In [119]:
import pandas as pd
from tqdm import tqdm
from itables import show as table_show

In [120]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [121]:
# obtention des donnnées de la CAN :
directory = '~/Bureau/Exports/20240513/non_anonyme/'
file = 'AGS_20240513_exports_agronomes_parcelles_non_rattachees.csv'
df_can = pd.read_csv(directory+file, sep='@')

In [122]:
df_can['domaine_id'].value_counts()

fr.inra.agrosyst.api.entities.Domain_112c6f9f-d37b-445f-b8f8-943962e4ceb4    1
fr.inra.agrosyst.api.entities.Domain_d473bf5e-6982-4491-ad4b-b8ce7ce8fe90    1
fr.inra.agrosyst.api.entities.Domain_a65b7438-f310-463d-8739-4ff09597e8aa    1
fr.inra.agrosyst.api.entities.Domain_55245350-6e07-4e73-8719-24e4e296d6a6    1
fr.inra.agrosyst.api.entities.Domain_359ecd15-6ad4-41a3-bce8-5fee9734ca97    1
                                                                            ..
fr.inra.agrosyst.api.entities.Domain_139588e2-e547-4343-a848-5801b93393ba    1
fr.inra.agrosyst.api.entities.Domain_e774acbe-cd41-4822-b9eb-bdbbf1f4937c    1
fr.inra.agrosyst.api.entities.Domain_98093cfd-5b13-429e-978e-b998c0997b01    1
fr.inra.agrosyst.api.entities.Domain_196b90ab-7533-40da-9225-b5902bdaa9d3    1
fr.inra.agrosyst.api.entities.Domain_801aa2e4-41c4-40e1-b777-82daafb2b736    1
Name: domaine_id, Length: 553, dtype: int64

In [123]:
donnees = {}

def import_df(df_name, path_data, sep, index_col=None):
    donnees[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
   'domaine', 'sdc', 'dispositif', 'parcelle', 'reseau'
]

tables_liaison = [
    'liaison_reseaux', 'liaison_sdc_reseau', 'liaison_reseaux'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)
import_dfs(tables_liaison, ENTREPOT_PATH, sep = ',', verbose=False)


100%|██████████| 3/3 [00:00<00:00, 76.76it/s]


In [124]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.Domain_0c482108-2711-43fe-9eed-ff2b8a545223',
    'fr.inra.agrosyst.api.entities.Domain_6b95065a-ecb3-4b83-8b2e-f5f4aa73030d',
    'fr.inra.agrosyst.api.entities.Domain_f63e841c-0db6-4c4f-800e-62e09ab096f8',
    'fr.inra.agrosyst.api.entities.Domain_6f9b9100-7d39-4980-a117-9b01dd0538f5',
    'fr.inra.agrosyst.api.entities.Domain_e87a7c0c-f743-4a6b-a7a4-45f32c0dbb3a'
]

In [125]:
donnees['parcelle'] = donnees['parcelle'].loc[
    donnees['parcelle']['domaine_id'].isin(studied_ids)
]
donnees['dispositif'] = donnees['dispositif'].loc[
    donnees['dispositif']['domaine_id'].isin(studied_ids)
]
donnees['sdc'] = donnees['sdc'].loc[
    donnees['sdc']['dispositif_id'].isin(list(donnees['dispositif'].index))
]
donnees['liaison_sdc_reseau'] = donnees['liaison_sdc_reseau'].loc[
    donnees['liaison_sdc_reseau']['sdc_id'].isin(donnees['sdc'].index)
]
donnees['liaison_reseaux'].loc[
    donnees['liaison_reseaux']['reseau_id'].isin(donnees['liaison_sdc_reseau']['reseau_id'])
]
donnees['reseau'] = donnees['reseau'].loc[
    donnees['reseau'].index.isin(donnees['liaison_reseaux']['reseau_id']) | 
    donnees['reseau'].index.isin(donnees['liaison_reseaux']['reseau_parent_id'])
]

In [137]:
pwd

'/home/bvuittenez/Bureau/utils/agrosyst_entrepot/utils/entrepot_nettoyage/catalogue_script_agrosyst/tests/data/test_get_parcelles_non_rattachees_outils_can'

In [138]:
path='./'
donnees['parcelle'].to_csv(path+'parcelle'+'.csv')
donnees['dispositif'].to_csv(path+'dispositif'+'.csv')
donnees['sdc'].to_csv(path+'sdc'+'.csv')
donnees['liaison_sdc_reseau'].to_csv(path+'liaison_sdc_reseau'+'.csv')
donnees['liaison_reseaux'].to_csv(path+'liaison_reseaux'+'.csv')
donnees['reseau'].to_csv('../../../data/referentiels/reseau'+'.csv')

In [134]:
!rm ./reseau.csv.gpg
!gpg --recipient berenger.vuittenez@inrae.fr --encrypt --output ./reseau.csv.gpg ./reseau.csv
# par la suite, penser à la déplacer le fichier produit dans référentiel car c'est là qu'il sera utilisé pour le test


In [135]:
!rm ./reseau.csv